# Importacion de librerias

In [ ]:
import pandas as pd
from Validaciones import Validador, distancia, verificaciones_sumas, agrupacion_de_bases, score_str_vs_str
import re 
import sqlite3


rfc_regex=re.compile("^\w\w\w\w?\d\d\d\d\d\d")

# Carga de archivos

In [ ]:
Municipio = 'GEM'  # Naucalpan  Ecatepec  Huixquilucan

In [ ]:
# path = "C:/Users/52551/Documents/Trabajo/Equipo_Agua/Issemym/Data/cfdis/Nov 2022 cfdis_Ecatepec.xlsx" 
# path = "C:/Users/52551/Documents/Trabajo/Equipo_Agua/Issemym/Data/cfdis/Naucalpan_cfdis_Noviembre.xlsx"
# path = "C:/Users/52551/Documents/Trabajo/Equipo_Agua/Issemym/Data/cfdis/CFDIS HUIXQUILUCAN.xlsx"
# path=r"G:\Mi unidad\Equipo_Agua\Issemym\Data\CFDI_ultimoproceso.csv"
path=r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce\CFDI_ultimoproceso.csv"
path=r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce\CFDI_Septiembre_2022\CFDI_Septiembre_2022_importante.xlsx"
df_cfdi = pd.read_excel(path)
df_cfdi["RfcReceptor"]=df_cfdi["cfdi_Receptor_Rfc"]
df_cfdi
len(df_cfdi)



In [ ]:
# con=sqlite3.connect("G:\Mi unidad\Equipo_Agua\Issemym\Data\sql\salida_nombre")
con=sqlite3.connect(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\salida_nombre")
base=pd.read_sql_query("SELECT RfcReceptor,NombreRazonSocialReceptor FROM salida",con=con)
con.close()
base.columns=["RfcReceptor",'NombreRazonSocialReceptor']

In [ ]:
base1=pd.read_excel(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce\RFC_Razonsocial_cfdi.xlsx",index_col=0)
base1.columns=["RfcReceptor",'NombreRazonSocialReceptor']
base=pd.concat([base,base1],ignore_index=True).drop_duplicates()
base=base.drop_duplicates(subset="RfcReceptor")
base["NombreRazonSocialReceptor"]=base["NombreRazonSocialReceptor"].str.replace("_"," ")

In [ ]:
base.columns=["RfcReceptor",'NombreRazonSocialReceptor']
# df_cfdi=pd.merge(df_cfdi,base,on="RfcReceptor")


In [ ]:
df_cfdi['NombreRazonSocialReceptor']=[str(x).strip().replace("_"," ").replace("Ã‘","Ñ") for x in df_cfdi["Nombre"]]

In [ ]:
# df_cfdi = df_cfdi[(df_cfdi["EfectoComprobante"]=="Nómina")&(df_cfdi["EstadoComprobante"]=="Vigente")]
try:
    df_cfdi['key'] = [rfc_regex.findall(x)[0] for x in df_cfdi["RfcReceptor"]]
except:
    keys=[]
    for x in df_cfdi["RfcReceptor"]:
        try:
            regex1=rfc_regex.findall(x)
        except:
            regex1=[]
        if len(regex1)>0:
            keys.append(regex1[0])
        else:
            keys.append("0")
    df_cfdi['key']=keys
v1 = Validador(df_cfdi, Municipio)
v1.Quita_caracteres(columns=['NombreRazonSocialReceptor'])
v1.Quitar_acentos(columns=['NombreRazonSocialReceptor'])
df_cfdi['Full name'] = df_cfdi['NombreRazonSocialReceptor'].str.strip()
df_cfdi = v1.df1
df_cfdi.shape, df_cfdi.columns

In [ ]:
def preparativos_issemym(df_issemym:pd.DataFrame, Municipio:str, save_folder:str, Nombre_base:str='Issemym'):
    '''
    (Function)
        Funcion que hace preparativos para el archivo Issemym
    (Paramaters)
        -path: Direccion path, donde se encuentra alojado el archivo
        -Municipio: Municipio por el cual se filtrara la base, recuerde que en este momento solo hay 3 municipios
        -Nombre_base: Nombre con el cual se creara el objeto validador
    (Return)
        DataFrame
    '''
    # df_issemym = df_issemym.loc[df_issemym['Municipio']==Municipio]
    try:
        df_issemym['key'] = [rfc_regex.findall(x)[0] for x in df_issemym["CURP"]]
    except:
        keys=[]
        for x in df_issemym["CURP"]:
            try:
                regex1=rfc_regex.findall(x)
            except:
                regex1=[]
            if len(regex1)>0:
                keys.append(regex1[0])
            else:
                keys.append("0")
    df_issemym["key"]=keys
    df_issemym["Conteo_issemym".upper()] = 1
    df_issemym['Full name'] = df_issemym['NOMBRE'].fillna('').replace(" ","_") #+ ' '+ df_issemym['Apellido Paterno'].fillna('') + ' '+ df_issemym['Apellido Materno'].fillna('')
    df_issemym['Full name'] = df_issemym['Full name'].str.strip()
    v2 = Validador(df_issemym,Nombre_base)
    v2.Quita_caracteres(columns=['Full name','NOMBRE'])
    v2.Quitar_acentos(columns=['Full name'])
    issemym_rfc_invalido = v2.panorama_grl(tipo='excel', save_folder=save_folder)
    df_issemym = v2.df1
    return df_issemym, v2, issemym_rfc_invalido
    df_issemym.shape, df_issemym.columns #33952

In [ ]:
# path = r"G:\Mi unidad\Equipo_Agua\Issemym\Data\issemym_cruce_202217_rapido.csv"
path=r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce\issemym_cruce_202217_rapido.csv"
df_issemym = pd.read_csv(path)



In [ ]:
a=df_cfdi[["RfcReceptor","Full name"]].drop_duplicates()
a[a["Full name"].duplicated()].sort_values(by="Full name")

In [ ]:
df_issemym.columns

In [ ]:
# df_issemym.to_csv(r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce\issemym_cruce_202217_rapido.csv")

In [ ]:
# save_folder=r"G:\Mi unidad\Equipo_Agua\Issemym\salidas"
save_folder=r"C:\Users\ASUS\Documents\GobiernoEdomex\Agua\GEM\cruce/"
df_issemym, v2, issemym_rfc_invalido = preparativos_issemym(df_issemym,Municipio, save_folder=save_folder)

## Agrupacion de datos

In [ ]:
redifinir_tipo_cols = ['Percepciones Cotizables'.upper()]
for col in redifinir_tipo_cols:
    df_issemym[col] = df_issemym[col].astype(float)

In [ ]:
df_issemym.info()

# Pre-procesing de datos

## Vision general
ya hemos instanciado el Validador, para la base cfdi se asocio la variable ***v1***, y para la base Issemym se asocio ***v2***

In [ ]:

cfdi_rfc_invalido = v1.panorama_grl(tipo='excel', save_folder=save_folder)

In [ ]:
issemym_rfc_invalido, cfdi_rfc_invalido

Una vez estandarizados las columnas pertinentes haremos un agrupado como para el cruce, con la finalidad de detectar keys repetidas y/o rfc y/o curps. Por las pruebas que se han hecho, se recomienda resolver todos los inconvenientes que saldran a continuacion, antes de hacer el cruce, ya que de lo contrario cabe la proabibilidad (mas alta) de que los montos no cuadren

In [ ]:
df_issemym['Clave ISSSEMYM'] = df_issemym["GEMPERSONACVEISSEMYM"]
dict_issemym_agg = {"CONTEO_ISSEMYM":"count","PERCEPCIONES COTIZABLES":sum,'IMPORTE DE CUOTA DE SALUD':sum,
 'IMPORTE DE CUOTA SOLIDARIO':sum, 'IMPORTE DE APORTACIÓN SOLIDARIO':sum, 'IMPORTE DE APORTACIÓN GASTOS DE ADMINISTRACIÓN':sum,
 'IMPORTE DE APORTACIÓN PRIMA BÁSICA':sum, 'IMPORTE DE APORTACIÓN DE PRIMA POR SINIESTROS':sum, 'IMPORTE DE APORTACIÓN DE PRIMA POR RIESGOS':sum}
columnas=[]
for col in df_issemym.columns:
    col=col.strip()
    columnas.append(col)
df_issemym.columns=columnas

In [ ]:
# df_cfdi["Salario Base Cotizacion"].head(5)

In [ ]:
df_cfdi["RfcEmisor"] = 1
# df_cfdi["Subtotal"]=df_cfdi["Salario Base Cotizacion"]

In [ ]:
df_cfdi_agg, df_issemym_agg = agrupacion_de_bases(df_cfdi, df_issemym,dict_issemym_agg=dict_issemym_agg,dict_cfdi_agg={"SubTotal":sum})

In [ ]:
df_cfdi_agg["Full name"].value_counts()

## Revision de duplicados

In [ ]:
rfc_cfdi_duplicada = df_cfdi_agg.loc[df_cfdi_agg.RfcReceptor.duplicated()]['RfcReceptor'].values
key_cfdi_duplicada = df_cfdi_agg.loc[df_cfdi_agg.key.duplicated()]['key'].values
rz_cfdi_duplicada = df_cfdi_agg.loc[df_cfdi_agg.NombreRazonSocialReceptor.duplicated()]['NombreRazonSocialReceptor'].values

rfc_issemym_duplicada = df_issemym_agg.loc[df_issemym_agg['CURP'].duplicated()]['CURP'].values
rs_issemym_duplicada = df_issemym_agg.loc[df_issemym_agg['Full name'].duplicated()]['Full name']
key_issmym_duplicada = df_issemym_agg.loc[df_issemym_agg['key'].duplicated()]['key']


##### Razones sociales duplicadas

In [ ]:
df_cfdi_agg[(df_cfdi_agg['NombreRazonSocialReceptor'].isin(rz_cfdi_duplicada))][['NombreRazonSocialReceptor','RfcReceptor','key']].sort_values('NombreRazonSocialReceptor')

In [ ]:
df_issemym_agg[df_issemym_agg['Full name'].isin(rs_issemym_duplicada)].sort_values('Full name')

¿REperesentan un problema?   No porque las llaves son totalmente diferentes, pero en caso de tener una llave igual habria que revisar, igual se vera en el siguiente apartado 

##### RFCs duplicados

In [ ]:
print(len(rfc_cfdi_duplicada))
df_cfdi_agg[df_cfdi_agg['RfcReceptor'].isin(rfc_cfdi_duplicada)].sort_values('RfcReceptor')

In [ ]:
print(len(rfc_issemym_duplicada))
df_issemym_agg[df_issemym_agg['CURP'].isin(rfc_issemym_duplicada)]

¿Representan algun problema?           Claro que si, no puede haber mas de 2 personas con el mismo rfc, revisa porque se estan duplicando y corrigelo

##### Keys duplicados

In [ ]:
df_cfdi_agg[df_cfdi_agg['key'].isin(key_cfdi_duplicada)].sort_values('key')

In [ ]:
print(len(key_issmym_duplicada))
df_issemym_agg[df_issemym_agg['key'].isin(key_issmym_duplicada)]

¿Representan un problema?
No necesariamente una key repetida es problema, pues puede darse el caso en que dos personas tengan la misma key, pero sin embargo la razon social sera diferente, en tal caso, eso no representa un problema, porque hacemos el cruce por dos columnas (key y RazonSocial) Sin embargo debe cuidar el caso en que tengan nombre social y key igual, pero RFC diferente ya que son personas totalmente diferentes.

## Solucion de duplicados

La mejor manera es detectar una manera sistematica de solucionar estos errores, en este momento eh detectado un rfc con dos nombres en el cfdi


In [ ]:
Municipio

In [ ]:
# Para el caso de Ecatepec 
df_cfdi.loc[df_cfdi['key']=='MARE870706', 'NombreRazonSocialReceptor'] = 'ERICK JAEN MARTINEZ RETANA'

In [ ]:
# Para el caso de Naucalpan
df_cfdi['NombreRazonSocialReceptor'].loc[df_cfdi['NombreRazonSocialReceptor']=='JOSE GUADALUPE ROA FONSECA'    ] ='ROA FONSECA JOSE GUADALUPE'
df_cfdi['NombreRazonSocialReceptor'].loc[df_cfdi['NombreRazonSocialReceptor']=='NEREA NORMA VIDAL OBREGON'    ]  ='VIDAL OBREGON NEREA NORMA'
df_cfdi['NombreRazonSocialReceptor'].loc[df_cfdi['NombreRazonSocialReceptor']=='GUILLERMO JAVIER JIMENEZ CERDA']='JIMENEZ CERDA GUILLERMO JAVIER'
df_cfdi['NombreRazonSocialReceptor'].loc[df_cfdi['NombreRazonSocialReceptor']=='GABRIELA MORALES GUZMAN'     ]   ='MORALES GUZMAN GABRIELA'

In [ ]:
# Huixquilucan no hay

In [ ]:
df_cfdi_agg

## Reagrupacion una vez solucianado todo

In [ ]:
#Diccionario Deducciones GEM  #"C111":sum,
# dict_gem_deduc={"C108":sum,"C109":sum,"C110":sum,"C111":sum,"C112":sum,"C113":sum,"C114":sum,"C115":sum,"C116":sum,"C117":sum,"C118":sum,"Subtotal":"sum","RfcEmisor":"count"}
dict_gem_deduc={"SubTotal":sum}

In [ ]:
# df_cfdi_agg, df_issemym_agg = agrupacion_de_bases(df_cfdi, df_issemym)
df_issemym['RfcReceptor'] = df_issemym['CURP']
df_cfdi_agg, df_issemym_agg = agrupacion_de_bases(df_cfdi, df_issemym,cols_cfdi_agg=['NombreRazonSocialReceptor', "RfcReceptor"],dict_issemym_agg=dict_issemym_agg,
    cols_issemym_agg=['RfcReceptor',"Full name","key"], dict_cfdi_agg=dict_gem_deduc)

In [ ]:
i='YAVR670929PA3'
df_cfdi[df_cfdi["RfcReceptor"]==i]


In [ ]:
# df_cfdi_agg['C111'] = df_cfdi_agg['C111'].astype('float64', errors='ignore')
# df_cfdi_agg.info()

# Cruce de datos

In [ ]:
# df_issemym_agg["Full name"]=df_issemym_agg["Full name"].str.replace(" ","_")
df_cfdi_agg["Full name"]=df_cfdi_agg["Full name"].str.replace("_"," ")
cruce0 = df_cfdi_agg.merge(df_issemym_agg, on=["Full name",'RfcReceptor'], how='outer', indicator=True,suffixes=["","_issemym"])
cruce0

In [ ]:
cruce0['_merge'].value_counts()

In [ ]:
# cruce0.to_excel("cruce01.xlsx")

In [ ]:
# #Diccionario percepciones GEM 
# dict_Gem_percep_agg:dict={"C1":["C5", "C6","C7","C8","C9","C24","C36","C41","C43","C44","C51","C61","C66","C67","C68","C69","C70",
#                                 "C71","C72","C75","C76","C78","C82","C83","C84","C85","C88","C89","C92","C93","C94","C95","C96","C97",
#                                 "C98","C99","C100","C101","C102","C103","C105","C106"], 
#                           "C2":["C11"],
#                           "C3":["C16","C17"],
#                           "C4":["C54","C45","C45","C18","C46","C56","C57","C19","C20","C58","C21","C49",
#                           "C42","C59","C60","C22","C23","C25","C39","C40","C26","C27","C28","C29","C30",
#                           "C31","C32","C33","C62","C34","C81","C80","C63","C107","C35","C64","C65","C53","C55",
#                           "C37","C38","C90","C104","C91","C87","C47","C86","C52","C48","C50","C77","C73","C74","C79"]}

In [ ]:
# df_cfdi['C3']

In [ ]:
# #Suma de claves SAT
# for k, v in dict_Gem_percep_agg.items():
#     print(k,v)
#     for col in v:
#         df_cfdi[k]+=df_cfdi[col]

In [ ]:
cruce0[cruce0[col_key]=="MANA710215VD1"]


In [ ]:
ini = 0
fin = 10
for i in range(15):
    print(df_cfdi.columns[ini:fin])
    ini = fin
    fin += 10

In [ ]:
df_cfdi.shape

# Comprobacion de datos

## Suma de importes vertical

In [ ]:
col_verificaciones_issemym=list(dict_issemym_agg.keys())#['CONTEO_ISSEMYM', 'Percepciones Cotizables', 'Importe de Cuota SCI', 'Importe Aportación SCI', 'Importe de Cuota SCI Voluntaria', 'Importe de Aportación SCI Voluntaria', 'Importe de Cuota de SaludImporte de Aportación de Salud', 'Importe de Cuota Solidario', 'Importe de Aportación Solidario', 'Importe de Retención Institucional', 'Importe de Retención', 'Importe de Cuota Extraordinaria', 'Importe de Aportación Extraordinaria', 'Importe de Aportación Gastos de Administración', 'Importe de Aportación Prima Básica', 'Importe de Aportación Prima de Servicios', 'Importe de Aportación de Prima Por Siniestros ', 'Importe de Aportación de Prima por Riesgos']
col_verificaciones_issemym=[str(x).upper().strip() for x in col_verificaciones_issemym]
verificaciones_sumas(df_cfdi,df_issemym_agg,cruce0, col_verificaciones_issemym=col_verificaciones_issemym, 
col_verificaciones_cfdi=["SubTotal"])#'C108', 'C109',
#        'C110', 'C112', 'C113', 'C114', 'C115', 'C116', 'C117', 'C118',
#        'Subtotal'])

In [ ]:
df_cfdi["SubTotal"].sum()-cruce0["SubTotal"].sum()

In [ ]:
cruce0["_merge"].value_counts()

In [ ]:
cruce0.to_excel("primerquincena_resultado_cruce.xlsx")

## Re-validacion de datos

En este punto ya hemos corrobado la informacion de importes, pero ahora es importante revisar que no hay key duplicadas, puesto que todas debieron haber cruzado solo para un registro (Aunque puede darse el caso que dos personas coincidan, pero si los nombres son totalmente diferentes, no debe haber problema)

In [ ]:
Municipio

#### Duplicados por key

In [ ]:
col_key = 'RfcReceptor'
key_cruce_duplicada = cruce0[cruce0[col_key].duplicated()][col_key]
cruce0_key_repetida = cruce0[(cruce0[col_key]!="0")&((cruce0[col_key].isin(key_cruce_duplicada)) & (cruce0['_merge']!='both'))].sort_values(col_key).fillna('Vacio')
cruce0_key_repetida.head(6)  #30

In [ ]:
# cruce0[cruce0["Full name"].str.contains("nan")]

In [ ]:
cruce0[cruce0['RfcReceptor']=='YAVR670929PA3']

#### Duplicados por nombre

In [ ]:
rs_cruce_duplicada = cruce0[cruce0['Full name'].duplicated()]['Full name']
cruce0_rs_repetida = cruce0[(cruce0['Full name'].isin(rs_cruce_duplicada)) & (cruce0['_merge']!='both')].sort_values('Full name').fillna('Vacio')
cruce0_rs_repetida

In [ ]:
cruce0_rs_repetida[cruce0_rs_repetida[col_key]=="AOCJ900105000"]

### Solucion de anomalias por key repetida en el cruce

In [ ]:
import numpy as np
keyss=[]
cont = 0
for key in cruce0_key_repetida[col_key].unique():
    try:
        rs_cfdi = cruce0_key_repetida.loc[(cruce0_key_repetida[col_key]==key)&(cruce0_key_repetida['NombreRazonSocialReceptor']!='Vacio'), 'NombreRazonSocialReceptor'].values
        rs_issemym = cruce0_key_repetida.loc[(cruce0_key_repetida[col_key]==key)&(cruce0_key_repetida['NombreRazonSocialReceptor']=='Vacio'), 'Nombre Completo Issemym'].values
        # print(rs_issemym)
    # except:pass
        score_d = distancia(rs_issemym,rs_cfdi)
        # rs_cfdi_acotado = [rs_cfdi[0][:40]]
        # rs_issemym_acotado = [rs_issemym[0][:40]]
        # print(rs_issemym)
        # print(rs_cfdi)
        rs_cfdi1=[rs_cfdi[0][:40]]
        rs_cfdi1.append(str(rs_cfdi1[0]).replace("Ñ","N"))
        rs_issemym1=[rs_issemym[0][:40]]
        score_d = distancia(rs_issemym1,rs_cfdi1)
        if score_d['score_sort'].values[0] > 95 or score_d['score_sort'].values[1] > 95:
            print('Cambiar nombre del cfdi por el de Issemym')
            print(score_d['score_sort'])
            print(rs_cfdi[0], ' --> ',rs_issemym[0] )
            df_cfdi.loc[df_cfdi['NombreRazonSocialReceptor']==rs_cfdi[0], 'NombreRazonSocialReceptor'] = rs_issemym[0]
            cont += 1

        else:
            ans = input(f'Se comparo {rs_cfdi[0]} vs {rs_issemym[0]}, asumiendo llaves iguales ({score_d["score_sort"]}). Asumismo que es la misma persona? (1:si, 0:no)')
            if ans == '1':
                print('Cambiar nombre del cfdi por el de Issemym')
                print(score_d['score_sort'])
                print(rs_cfdi[0], ' --> ',rs_issemym[0] )
                df_cfdi.loc[df_cfdi['NombreRazonSocialReceptor']==rs_cfdi[0], 'NombreRazonSocialReceptor'] = rs_issemym[0]
                cont += 1
            elif ans.lower() == 'break':
                break
            else:
                print('Se omitio')

    except Exception as e:
        keyss.append(key)
        print(key, ' No valido  ', e)
        continue
    print('--'.center(45,'-'))
print('Se hicieron ', cont, ' cambios') 





In [ ]:
for i in keyss:
    df_cfdi.loc[df_cfdi[col_key]==i,"Full name"]=df_issemym[df_issemym[col_key]==i]["Full name"].values[0]
    df_cfdi.loc[df_cfdi[col_key]==i,"Nombre"]=df_issemym[df_issemym[col_key]==i]["Full name"].values[0]
    df_cfdi.loc[df_cfdi[col_key]==i,"NombreRazonSocialReceptor"]=df_issemym[df_issemym[col_key]==i]["Full name"].values[0]
    

In [ ]:
df_cfdi.loc[df_cfdi[col_key]==i]

In [ ]:
key
# cruce0_key_repetida.loc[(cruce0_key_repetida[col_key]==key)&(cruce0_key_repetida['NombreRazonSocialReceptor']!='Vacio'), 'NombreRazonSocialReceptor'].values

In [ ]:
# df_cfdi[df_cfdi["ID"]=='AAAR730221294']

In [ ]:
len("SANTILLAN MARTINEZ MARIA DE LA SOLEDAD S")

### Solucion de anomalias por Razon social
En caso de tener este tipo de anomalias, debemos ser cuidadosos, pues debemos revisar la fecha de nacimiento que sean las mismas, pues si tenemos nombres reptidos, es por que la key no es la misma (atribuible a un CURP o RFC mal capturado)

In [ ]:
fecha_regex=re.compile("\d\d\d\d\d\d")
cont = 0
for rs in cruce0_rs_repetida['Full name'].unique():
    try:
        key_cfdi    = cruce0_rs_repetida.loc[(cruce0_rs_repetida['Full name']==rs)&(cruce0_rs_repetida['Nombre Completo Issemym']!='Vacio'), col_key].values
        key_issemym = cruce0_rs_repetida.loc[(cruce0_rs_repetida['Full name']==rs)&(cruce0_rs_repetida['Nombre Completo Issemym']=='Vacio'), col_key].values
        # print(key_cfdi,key_issemym)
        key_cfdi=key_cfdi[0]
        key_issemym=key_issemym[0]
        key_cfdi_n    = fecha_regex.findall(key_cfdi)[0]
        key_issemym_n = fecha_regex.findall(key_issemym)[0]
        
        score_d = score_str_vs_str(key_cfdi_n,key_issemym_n, rapido=False)
        

        if score_d >95:
            print('Cambiar key del cfdi por el de Issemym')
            print(key_cfdi, ' --> ',key_issemym )
            print(score_d)
            df_cfdi.loc[df_cfdi[col_key]==key_cfdi, col_key] = key_issemym
            df_issemym.loc[df_issemym[col_key]==key_cfdi, col_key] = key_issemym
            cruce0_rs_repetida.loc[cruce0_rs_repetida[col_key]==key_cfdi,col_key]=key_issemym
            cont += 1
            
    except Exception as e:
        print(rs, ' No valido  ', e)
        continue
print('Se hicieron ', cont, ' cambios')        
            

In [ ]:
# ñ
display(df_cfdi[df_cfdi[col_key]=="ZUVA861215TJ0"])
df_issemym[df_issemym['Full name']==rs]
df_issemym_agg
# cruce0_rs_repetida.loc[(cruce0_rs_repetida['Full name']==rs)&(cruce0_rs_repetida['Nombre Completo Issemym']=='Vacio'), col_key].values
# cruce0_rs_repetida.loc[cruce0_rs_repetida[col_key]=="ZUVA861215000","RfcReceptor"]="ZUVA861215TJ0"
# cruce0_rs_repetida[cruce0_rs_repetida[col_key]=="ZUVA861215TJ0"]

In [ ]:
key_cfdi

In [ ]:
ñ

In [ ]:
print("RfcReceptor",col_key)

En caso de haber tenido que ejecutar las celdas anteriores, necesitara regresar hasta **Reagrupacion una vez solucianado todo**

### Etiquetar campos que no cruzaron 

In [ ]:
cruce0

In [ ]:
# por fechas diferentes, pese al nombre igual
cruce0.loc[cruce0_rs_repetida.index, 'Motivo de no Cruce'] = 'Con coincidencia pero Fechas diferentes en CURP y RFC'

In [ ]:
# Sin coincidencias 
col_tipo_cruce = '_merge'   # Tipo de cruce
excluir_tipo = 'both'             # Ambas
cruce0.loc[(cruce0[col_tipo_cruce]!=excluir_tipo) & (cruce0['Motivo de no Cruce'].fillna('Vacio')=='Vacio' ), 'Motivo de no Cruce'] = 'Sin coincidencias'

In [ ]:
cruce0

# Refinamiento

## Columnas extras

'ISSEMYM obligatorios' =[@[Suma de Percepciones Cotizables]]+[@[Suma de Importe de Cuota de Salud]]+[@[Suma de Importe de Aportación de Salud]]+[@[Suma de Importe de Cuota Solidario]]+[@[Suma de Importe de Aportación Solidario]]+[@[Suma de Importe de Aportación Gastos de Administración]]+[@[Suma de Importe de Aportación Prima Básica]]+[@[Suma de Importe de Aportación de Prima Por Siniestros ]]+[@[Suma de Importe de Aportación de Prima por Riesgos]]

In [ ]:
col_suma_issemym_oblig = ['Percepciones Cotizables','Importe de Cuota de Salud','Importe de Aportación de Salud','Importe de Cuota Solidario',
'Importe de Aportación Solidario','Importe de Aportación Gastos de Administración','Importe de Aportación Prima Básica','Importe de Aportación de Prima Por Siniestros ',
'Importe de Aportación de Prima por Riesgos']
cruce0['ISSEMYM obligatorios'] = 0
for col in col_suma_issemym_oblig:
    cruce0['ISSEMYM obligatorios'] += cruce0[col]

ISSEMyM otros importes = =[@[Suma de Importe Aportación SCI]]+[@[Suma de Importe de Cuota SCI]]+[@[Suma de Importe de Cuota SCI Voluntaria]]+[@[Suma de Importe de Aportación SCI Voluntaria]]+[@[Suma de Importe de Retención Institucional]]+[@[Suma de Importe de Retención]]+[@[Suma de Importe de Cuota Extraordinaria]]+[@[Suma de Importe de Aportación Extraordinaria]]+[@[Suma de Importe de Aportación Prima de Servicios]]

In [ ]:
col_suma_issemym_otros = ['Importe de Cuota SCI','Importe Aportación SCI','Importe de Cuota SCI Voluntaria','Importe de Retención Institucional',
'Importe de Retención','Importe de Cuota Extraordinaria', 'Importe de Aportación Extraordinaria','Importe de Aportación Prima de Servicios']
cruce0['ISSEMYM otros importes'] = 0
for col in col_suma_issemym_otros:
    cruce0['ISSEMYM otros importes'] += cruce0[col]

In [ ]:
cruce0['RFC_10_digitos'] = cruce0['key']
cruce0['Tipo de cruce'] = cruce0['_merge'].map({'both':'Ambas','left_only':'Solo_GEM','right_only':'Solo_Issemym'})

## Ordenar columnas

In [ ]:
cruce0 = cruce0[['Tipo de cruce','RFC_10_digitos','Motivo de no Cruce',
 'RfcReceptor',
 'NombreRazonSocialReceptor',
 'CURP',
 'Nombre Completo Issemym',
 'Full name',
 'Subtotal',
 'RfcEmisor',
 'Sueldo Neto',
 'Percepciones Cotizables',
 'Importe de Cuota SCI',
 'Importe Aportación SCI',
 'Importe de Cuota SCI Voluntaria',
 'Importe de Aportación SCI Voluntaria',
 'Importe de Cuota de Salud',
 'Importe de Aportación de Salud',
 'Importe de Cuota Solidario',
 'Importe de Aportación Solidario',
 'Importe de Retención Institucional',
 'Importe de Retención',
 'Importe de Cuota Extraordinaria',
 'Importe de Aportación Extraordinaria',
 'Importe de Aportación Gastos de Administración',
 'Importe de Aportación Prima Básica',
 'Importe de Aportación Prima de Servicios',
 'Importe de Aportación de Prima Por Siniestros ',
 'Importe de Aportación de Prima por Riesgos',
 'ISSEMYM obligatorios',
 'ISSEMYM otros importes']]

## Ultima validacion

In [ ]:
verificaciones_sumas(df_cfdi,df_issemym_agg,cruce0)

In [ ]:
cruce0['Motivo de no Cruce'].value_counts()

In [ ]:
cruce0['Tipo de cruce'].value_counts()

In [ ]:
nombre_archivo = f'Cruce_{Municipio}_Issemym_v4.xlsx'
nombre_archivo

In [ ]:
cruce0.to_excel(f'C:/Users/52551/Documents/Trabajo/Equipo_Agua/Issemym/Data/{nombre_archivo}', index=False)

# Conteo de registros

In [ ]:
cruce0[cruce0['Full name'].isin(cruce0[cruce0['Tipo de cruce'] == 'Solo_Issemym']['Full name'])].sort_values('Full name')